In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn_pandas import DataFrameMapper, gen_features, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.metrics import *
from scipy import stats
import xgboost as xgb

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
np.random.seed(10)

In [4]:
df = pd.read_pickle('data/train2.pkl')

In [5]:
df.columns

Index(['click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [7]:
df.nunique()

click                     2
hour                     24
C1                        7
banner_pos                7
site_id                2865
site_domain            3394
site_category            22
app_id                 4154
app_domain              287
app_category             31
device_id            368962
device_ip           1078153
device_model           6098
device_type               4
device_conn_type          4
C14                     785
C15                       8
C16                       9
C17                     193
C18                       4
C19                      44
C20                     168
C21                      38
dtype: int64

In [6]:
df['hour_of_day'] = df['hour'].map(lambda x: int(str(x)[-2:]))
# df['day'] = df['hour'].map(lambda x: int(str(x)[4:6]))
# df['month'] = df['hour'].map(lambda x: int(str(x)[2:4]))


In [7]:
# pick colums with less possible categories to keep things simple-- since it will be a very large matrix
# after one hot encoding

# df_cols = ['click', 'hour_of_day', 'C1', 'banner_pos',
#        'site_category', 'app_category',
#          'device_type', 'device_conn_type', 
#        'C15', 'C16', 'C17', 'C18']


df_cols = ['hour_of_day', 'C1', 'banner_pos', 'device_type', 'device_conn_type', 'C15', 'C16', 'C18', 'click']

feature_cols = ['hour_of_day', 'C1', 'banner_pos',
         'device_type', 'device_conn_type', 
       'C15', 'C16', 'C18']

df = df[df_cols]


In [8]:
df.head()

,hour_of_day,C1,banner_pos,device_type,device_conn_type,C15,C16,C18,click
4122995,0,1005,0,1,0,320,50,0,0
4122996,0,1005,0,1,0,320,50,1,0
4122997,0,1005,0,1,0,320,50,0,0
4122998,0,1005,0,1,0,320,50,3,1
4122999,0,1005,0,1,0,320,50,0,0


In [9]:
# mapper2 = DataFrameMapper([
#     (['C1'], LabelBinarizer()),
#     (['banner_pos'], LabelBinarizer()),
#     (['device_type'], LabelBinarizer()),
#     (['device_conn_type'], LabelBinarizer()),
#     (['C15'], LabelBinarizer()),
#     (['C16'], LabelBinarizer()),
#     (['C18'], LabelBinarizer()),
#     (['hour_of_day'], LabelBinarizer()),
#     (['day'], LabelBinarizer())
# ], input_df=True)


feature_def = gen_features(
    columns=feature_cols,
    classes=[LabelBinarizer])
# classes=[MinMaxScaler, LabelBinarizer])
    
mapper = DataFrameMapper(feature_def, input_df=True)
# feature_select = sklearn.feature_selection.SelectKBest(k=100)

In [10]:
X = df[feature_cols]
y = df.click

# for col in X.columns:
#     X[col].astype('str')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
X_train.head()

,hour_of_day,C1,banner_pos,device_type,device_conn_type,C15,C16,C18
7320253,12,1005,0,1,0,320,50,0
5272550,6,1005,0,1,0,320,50,3
8632696,18,1005,0,1,0,320,50,0
6533132,10,1005,0,1,0,320,50,0
8536846,17,1010,1,4,0,320,50,3


In [14]:

pipe_forest = Pipeline([
    ('featurize', mapper),
#     ('reduce_dim', KernelPCA(n_jobs=-1, kernel="rbf")),
    ('random_forest', RandomForestClassifier(n_jobs=-1, random_state=10, n_estimators=100))])

In [16]:
pipe_forest.fit(X_train, y_train)
y_prediction = pipe_forest.predict(X_test)
report_rf = classification_report( y_test, y_prediction )
print(report_rf)

             precision    recall  f1-score   support

          0       0.85      1.00      0.91   1124508
          1       0.54      0.02      0.04    209774

avg / total       0.80      0.84      0.78   1334282



In [ ]:
pipe_forest2 = Pipeline([
    ('featurize', mapper),
#     ('reduce_dim', KernelPCA(n_jobs=-1, kernel="rbf")),
    ('random_forest', RandomForestClassifier(n_jobs=-1, random_state=10, n_estimators=100, class_weight="balanced"))])
pipe_forest2.fit(X_train, y_train)
y_prediction = pipe_forest.predict(X_test)
report_rf = classification_report( y_test, y_prediction )
print(report_rf)

Unfortunately, as was seen before, the classifier did very badly in recall. Would it help to change the prediction threshold?

In [18]:
def threshold_prediction(clf, X, thres):
    return (clf.predict_proba(X)[:,1] > thres).astype(int)
def predict_proba_scoring(thres):
    def custom_predict(clf, X, y):
        y_predicted = threshold_prediction(clf, X, thres)
        return metrics.f1_score(y, y_predicted)
    return custom_predict

In [ ]:
pipe_forest.predict_proba(X_train)

In [ ]:
cv_test = []
for thres in np.arange(0.05, 0.85, 0.1):
    scoring = predict_proba_scoring(thres)
    cv_score = cross_val_score(pipe_forest, X_train, y_train, cv=3, scoring=scoring)
    print("cv score: {} for threshold val {}".format(cv_score, thres))
    cv_test.append(cv_score)

In [ ]:
from hyperopt import tpe, hp, STATUS_OK, Trials, fmin

params = {
    'random_forest__max_depth': hp.choice('max_depth', range(1,20)),
    'random_forest__max_features': hp.choice('max_features', range(1,150)),
    'random_forest__n_estimators': hp.choice('n_estimators', range(50,500)),
    'random_forest__criterion': hp.choice('criterion', ["gini", "entropy"])}

best = 0
def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    print ('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=100, trials=trials)
print ('best:')
print (best)
# Parameter search space
space = {}


# One of (1,1), (1,2), or (1,3)
space['vec__ngram_range'] = hp.choice('vec__ngram_range', [(1,1), (1,2), (1,3)])

# Random integer in [1,3]
space['vec__min_df'] = 1+hp.randint('vec__min_df', 3)

# Uniform between 0.7 and 1
space['vec__max_df'] = hp.uniform('vec__max_df', 0.7, 1.0)
    
# One of True or False
space['vec__sublinear_tf'] = hp.choice('vec__sublinear_tf', [True, False])
    
# Random number between 50 and 100
space['kbest__percentile'] = hp.uniform('kbest__percentile', 50, 100)
    
# Random number between 0 and 1
space['clf__l1_ratio'] = hp.uniform('clf__l1_ratio', 0.0, 1.0)
      
# Log-uniform between 1e-9 and 1e-4
space['clf__alpha'] = hp.loguniform('clf__alpha', -9*np.log(10), -4*np.log(10))
      
# Random integer in 20:5:80
space['clf__n_iter'] = 20 + 5*hp.randint('clf__n_iter', 12)

In [4]:
pipeline = Pipeline([
    ('featurize', featurize(features)), 
    ('forest', RandomForestClassifier(n_estimators = 10))
])

pipeline.fit(X, y).predict(df_train)

param_grid = dict(forest__n_estimators = [2, 16, 32], forest__criterion = ['gini', 'entropy'])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, scoring='accuracy')

best_pipeline = grid_search.fit(X, y).best_estimator_
best_pipeline.get_params()['forest']


NameError: name 'featurize' is not defined

In [29]:
X_train.shape

(36210029, 10)